In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
class Camera():
    def __init__(self):
        self.calibrateCamera()

    def calibrateCamera(self):
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

        nx = 9
        ny = 6

        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image plane.

        # Make a list of calibration images
        images = glob.glob('camera_cal/calibration*.jpg')

        # Step through the list and search for chessboard corners
        for fname in images:
            img = mpimg.imread(fname)
            self.img_size = (img.shape[0], img.shape[1])
            gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

            # If found, add object points, image points
            if ret == True:
                objpoints.append(objp)
                imgpoints.append(corners)

        ret, self.mtx, self.dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[0], img.shape[1]), None, None)

camera = Camera()

In [ ]:
from collections import deque
class Line():
    def __init__(self, camera):
        self.mtx = camera.mtx
        self.dist = camera.dist
        self.shouldCalibrate = True
        self.distanceHistory = deque()
        # Define conversions in x and y from pixels space to meters
        self.ym_per_pix = 30/720 # meters per pixel in y dimension
        self.xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    def appendDistance(self, d):
        self.distanceHistory.append(d)
        if(len(self.distanceHistory) > 6):
            self.distanceHistory.popleft()
    
    def averageDistance(self):
        avgDistance = np.mean(np.array(self.distanceHistory))
        if(len(self.distanceHistory) < 6):
            return -1
        return avgDistance

    def warpImage(self, img):
        self.img = img
        img_size = (img.shape[1], img.shape[0])
        self.ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
        pt1 = ((img_size[0] / 2) - 55, img_size[1] / 2 + 100)
        pt2 = ((img_size[0] / 6) - 10, img_size[1])
        pt3 = ((img_size[0] * 4 / 6) + 80, img_size[1])
        pt4 = (img_size[0] / 2 + 50, img_size[1] / 2 + 100)
        
        self.undst = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)

        src = np.float32(
            [[pt1[0], pt1[1]],
            [pt2[0], pt2[1]],
            [pt3[0], pt3[1]],
            [pt4[0], pt4[1]]])
        dst = np.float32(
            [[(img_size[0] / 4) - 100, -500],
            [(img_size[0] / 4) - 100, img_size[1]],
            [(img_size[0] * 3 / 4), img_size[1]],
            [(img_size[0] * 3 / 4), -500]])

        M = cv2.getPerspectiveTransform(src, dst)
        self.Minv = cv2.getPerspectiveTransform(dst, src)
        self.warped = cv2.warpPerspective(self.undst, M, (img_size[0], img_size[1]), flags=cv2.INTER_LINEAR)
        
        return self.warped
    
    def s_channel_filter(self, s_channel):
        s_thresh_min = 30
        s_thresh_max = 255
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
        if(np.sum(s_binary[:,0:50]) > 9200 or np.sum(s_binary[:,1230:1280]) > 2400):
            s_thresh_min = 100
            s_thresh_max = 255
            s_binary = np.zeros_like(s_channel)
            s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

        return s_binary

    def l_channel_filter(self, l_channel):
        l_thresh_min = 200
        l_thresh_max = 255
        l_binary = np.zeros_like(l_channel)
        l_binary[(l_channel >= l_thresh_min) & (l_channel <= l_thresh_max)] = 1

        return l_binary

    def sx_filter(self):
        sx_thresh_min = 30
        sx_thresh_max = 255
        gray = cv2.cvtColor(self.warped,cv2.COLOR_RGB2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        abs_sobelx = np.absolute(sobelx)
        scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
        sx_binary = np.zeros_like(scaled_sobel)
        sx_binary[(scaled_sobel >= sx_thresh_min) & (scaled_sobel <= sx_thresh_max)] = 1

        return sx_binary

    def color_filter(self):
        hsv = cv2.cvtColor(self.warped, cv2.COLOR_RGB2HSV)
        lower = np.array([0,0,170])
        upper = np.array([255,255,255])
        mask = cv2.inRange(hsv, lower, upper)
        res = cv2.bitwise_and(self.warped,self.warped, mask= mask)
        gray = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY)
        color_binary = np.zeros_like(gray)
        color_binary[(gray >= 10) & (gray <= 255)] = 1
        if(np.sum(color_binary[:,5:10]) > 2000):
            color_binary = np.zeros_like(gray)
            color_binary[(gray >= 10) & (gray <= 155)] = 1
        return color_binary

    def filtered_with_color(self):
        hls = cv2.cvtColor(self.warped, cv2.COLOR_RGB2HLS)
        h_channel = hls[:,:,0]
        l_channel = hls[:,:,1]
        s_channel = hls[:,:,2]

        self.s_binary = self.s_channel_filter(s_channel)
        self.l_binary = self.l_channel_filter(l_channel)
        color_binary = self.color_filter()

        # Combine the two binary thresholds
        self.combined_binary = np.zeros_like(l_channel)
        self.combined_binary[(color_binary == 1)|(self.s_binary == 1)|(self.l_binary == 1)] = 1

        self.combined_binary[0:50,:]=0
        self.combined_binary[:,0:50]=0
        return self.combined_binary

    def filtered_with_sx(self):
        sx_binary = self.sx_filter()
        
        # Combine the two binary thresholds
        self.combined_binary = np.zeros_like(self.l_binary)
        self.combined_binary[(sx_binary == 1)|(self.s_binary == 1)|(self.l_binary == 1)] = 1

        self.combined_binary[0:50,:]=0
        self.combined_binary[:,0:50]=0
        return self.combined_binary

    def calculatePolyParams(self, left_x_values, left_y_values, right_x_values, right_y_values):
        left_fit_params = np.polyfit(left_y_values, left_x_values, 2)
        right_fit_params = np.polyfit(right_y_values, right_x_values, 2)

        left_fit_values = left_fit_params[0]*self.ploty**2 + left_fit_params[1]*self.ploty + left_fit_params[2]
        right_fit_values = right_fit_params[0]*self.ploty**2 + right_fit_params[1]*self.ploty + right_fit_params[2]
        return (left_fit_params, right_fit_params, left_fit_values, right_fit_values)

    def calculateRadius(self, left_x_values, right_x_values, y_eval):
        ym_per_pix = self.ym_per_pix
        xm_per_pix = self.xm_per_pix
        
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(self.ploty*ym_per_pix, left_x_values*xm_per_pix, 2)
        right_fit_cr = np.polyfit(self.ploty*ym_per_pix, right_x_values*xm_per_pix, 2)
        
        # Calculate the new radii of curvature
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
        return (left_curverad, right_curverad)

    def calculateCenterOffset(self, left_fit_values, right_fit_values):
        lane_center_x = ((right_fit_values[-1] + left_fit_values[-1])/2.0)*self.xm_per_pix
        midpoint_x = (self.img.shape[1]/2.0) * self.xm_per_pix

        center_offset = np.abs(lane_center_x - midpoint_x)
        return center_offset
        
    def calibrate(self):
        histogram = np.sum(self.combined_binary, axis=0)
        self.out_img_draw = out_img_draw = np.dstack((self.combined_binary, self.combined_binary, self.combined_binary)) * 255

        image_height = out_img_draw.shape[0]
        image_width = out_img_draw.shape[1]

        nwindow = 9
        rect_height = int(image_height/nwindow)
        margin = 100

        midpoint = histogram.shape[0]//2
        leftx_base = np.argmax(histogram[180:midpoint])+180
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        leftx_current = leftx_base
        rightx_current = rightx_base
        base_line = image_height

        leftx_values = np.array([])
        lefty_values = np.array([])
        rightx_values = np.array([])
        righty_values = np.array([])

        for i in range(nwindow):
            left = leftx_current - margin
            top = base_line - rect_height
            right = leftx_current + margin
            bottom = base_line
            rect = out_img_draw[top:bottom, left:right]
            x_values = rect.nonzero()[1]
            y_values = rect.nonzero()[0]
            leftx_values = np.append(leftx_values, x_values + left)
            lefty_values = np.append(lefty_values, y_values + base_line)
            if(len(x_values) > 10):
                leftx_current = int(np.mean(x_values)) + left

            left = rightx_current - margin
            right = rightx_current + margin
            rect = out_img_draw[top:bottom, left:right]
            x_values = rect.nonzero()[1]
            y_values = rect.nonzero()[0]
            rightx_values = np.append(rightx_values, x_values + left)
            righty_values = np.append(righty_values, y_values + base_line)
            if(len(x_values) > 10):
                rightx_current = int(np.mean(x_values)) + left

            base_line = top
        
        if(len(leftx_values) == 0 or len(rightx_values) == 0):
            return
        
        p = line.polyParamsAndRadius(leftx_values, lefty_values, rightx_values, righty_values)
        
        self.left_fit_values = p[0]
        self.right_fit_values = p[1]

    def findWithPreviousLane(self):
        nonzero = self.combined_binary.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (self.left_fit_params[0]*(nonzeroy**2) + self.left_fit_params[1]*nonzeroy + 
        self.left_fit_params[2] - margin)) & (nonzerox < (self.left_fit_params[0]*(nonzeroy**2) + 
        self.left_fit_params[1]*nonzeroy + self.left_fit_params[2] + margin))) 

        right_lane_inds = ((nonzerox > (self.right_fit_params[0]*(nonzeroy**2) + self.right_fit_params[1]*nonzeroy + 
        self.right_fit_params[2] - margin)) & (nonzerox < (self.right_fit_params[0]*(nonzeroy**2) + 
        self.right_fit_params[1]*nonzeroy + self.right_fit_params[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx_values = nonzerox[left_lane_inds]
        lefty_values = nonzeroy[left_lane_inds] 
        rightx_values = nonzerox[right_lane_inds]
        righty_values = nonzeroy[right_lane_inds]
       
        if(len(leftx_values) == 0 or len(rightx_values) == 0):
            return
        
        p = line.polyParamsAndRadius(leftx_values, lefty_values, rightx_values, righty_values)
        
        self.left_fit_values = (self.left_fit_values + p[0])/2
        self.right_fit_values = (self.right_fit_values + p[1])/2
    
    def polyParamsAndRadius(self, leftx_values, lefty_values, rightx_values, righty_values):
        p = line.calculatePolyParams(leftx_values, lefty_values, rightx_values, righty_values)
        left_fit_params = p[0]
        right_fit_params = p[1]
        left_fit_values = p[2]
        right_fit_values = p[3]
        
        r = line.calculateRadius(left_fit_values, right_fit_values, np.max(self.ploty))
        self.left_curverad = r[0]
        self.right_curverad = r[1]
        
        self.left_fit_params = left_fit_params
        self.right_fit_params = right_fit_params
        
        self.center_offset = self.calculateCenterOffset(left_fit_values, right_fit_values)
        
        return (left_fit_values, right_fit_values)
        
    def processImage(self):
        if(self.shouldCalibrate == True):
            line.calibrate()
            self.shouldCalibrate= False
        else:
            self.findWithPreviousLane()

    def finalResult(self):
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(self.combined_binary).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([self.left_fit_values, self.ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([self.right_fit_values, self.ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, self.Minv, (self.img.shape[1], self.img.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(self.undst, 1, newwarp, 0.9, 0)

        return result

In [ ]:
import os
files = os.listdir("test_images/")
for file in files:
    if(file.startswith('.')):
        continue
    img = mpimg.imread('test_images/' + file)
    line = Line(camera)
    w = line.warpImage(img)
    line.filtered_with_color()
    line.calibrate()
    result = line.finalResult()
    mpimg.imsave('output_images/' + file, result)

In [ ]:
line = Line(camera)

In [ ]:
class LineManager():
    def __init__(self, line):
        self.line = line
        self.i = 0
        self.right_fit_values=None
        self.left_fit_values=None

    def curveCheck(self):
        return self.line.left_curverad < 600 or self.line.right_curverad < 450

    def getDistance(self):
        return self.line.right_fit_values[0] - self.line.left_fit_values[0]

    def distanceCheck(self):
        distance = self.getDistance()
        avgDistance = self.line.averageDistance()
        if(avgDistance == -1):
            return True
        margin = 40
        return avgDistance - margin <= distance <= avgDistance + margin
    
    def processImage(self, img):
        status=""
        self.i+=1
        self.line.warpImage(img)
        if(self.i==1):
            self.line.filtered_with_color()
            self.line.calibrate()
            if(self.curveCheck()):
                self.line.filtered_with_sx()
                self.line.calibrate()
            result = self.line.finalResult()

            self.right_fit_values = self.line.right_fit_values
            self.left_fit_values = self.line.left_fit_values

            self.line.appendDistance(self.getDistance())
            return result
        
        self.line.filtered_with_color()
        self.line.processImage()
        status += "color->"
        if(self.curveCheck()):
            self.line.filtered_with_sx()
            self.line.processImage()
            status += "sx->"
        if(self.curveCheck()):
            w=self.line.filtered_with_color()
            self.line.calibrate()
            status += "color_calibrate->"
        if(self.curveCheck()):
            w=self.line.filtered_with_sx()
            self.line.calibrate()
            status += "sx_calibrate->"

        distance= self.getDistance()
        avgDistance = self.line.averageDistance()

        if(self.distanceCheck() == False):
            self.line.left_fit_values = self.left_fit_values
            self.line.right_fit_values = self.right_fit_values
            status += "distance->"
        if(self.curveCheck()):
            self.line.left_fit_values = self.left_fit_values
            self.line.right_fit_values = self.right_fit_values
            status += "curve->"

        self.right_fit_values = self.line.right_fit_values
        self.left_fit_values = self.line.left_fit_values

        result = self.line.finalResult()

        font = cv2.FONT_HERSHEY_SIMPLEX
        radius_str = "Radius of Curvature = %.2f (m)" % line.right_curverad
        vehicle_str = "Vehicle is %.2f m left of center" % line.center_offset
        cv2.putText(result,radius_str,(10,40), font, 1,(255,255,255),1,cv2.LINE_AA)
        cv2.putText(result,vehicle_str,(10,70), font, 1,(255,255,255),1,cv2.LINE_AA)
        
        #debugging
        #cv2.putText(result,status,(10,100), font, 1,(255,255,255),1,cv2.LINE_AA)
        
        self.line.appendDistance(self.getDistance())

        return result        


In [ ]:
line = Line(camera)
lineManager = LineManager(line)

def process_image(img):
    return lineManager.processImage(img)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'videoOutput/project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
line = Line(camera)
lineManager = LineManager(line)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'videoOutput/challenge_video.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)